In [2]:
#Does not work yet
raise Exception("Does not work yet")

Exception: Does not work yet

In [10]:
from datetime import datetime

import requests
import textwrap
def LLM(prompt, system_message, temperature, max_tokens, model,return_json=False):
    system_message = "Todays DateTime:"+datetime.now().strftime("%Y-%m-%d %H:00:00")+"\n"+system_message
    url = 'http://localhost:11435/api/passthrough/chat'
    headers = {'Content-Type': 'application/json'}
    payload = {
        'model': model,
        'cache': '1h',
        'keep_alive': '0m',
        'messages': [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': prompt}
        ],
        'temperature': temperature,
        'max_tokens': max_tokens,
        'return_json': return_json
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an HTTPError if the HTTP request returned an unsuccessful status code
        data = response.json()
        #print('Success:', data)
        return data
    except requests.exceptions.RequestException as error:
        print('Error:', error)
        raise




import json
def put_text_in_file(filename, text):
    """ Write text to a file in the jail directory """
    with open("./jail/" + filename, "w") as f:
        f.write(text)
    return "./jail/" + filename

import json
def create_directory(directory_name):
    """ Create a directory in the jail directory """
    import os
    os.makedirs("./jail/" + directory_name)
    return "./jail/" + directory_name


my_tools = [put_text_in_file, create_directory]
tools=[]
tool_matchings = {}
# use openai function to json module
for tool in my_tools:
    tools.append({
        "name": tool.__name__,
        "doc": tool.__doc__,
        "function_arguments": tool.__code__.co_varnames[:tool.__code__.co_argcount],
    })
    tool_matchings[tool.__name__] = tool
tools = json.dumps(tools, indent=2)
system_message="""

Answer the following questions as best you can. 
You have access to the following tools:
"""+tools+"""
Use them in parallel to solve the problem.

respond ONLY in multi item format of dictionaries in list
[
  {
    "tool": "name",
    "arguments": ["arg1", "arg2"]
  },
  {
    "tool": "name",
    "arguments": ["arg1", "arg2"]
  }
]
Valid Json Only
"""
reply = LLM("""

write me a python tick tack toe game with a gui in seperate files

""", system_message, 0.9, 1500, 'llama3:8b',return_json=True)
print('\n'.join(textwrap.wrap(reply['message']['content'], width=50)))
print(tool_matchings)
    
reply_json = json.loads(reply['message']['content'])
if type(reply_json) == list:
    for tool_call in reply_json:
        print(tool_call)
        tool = tool_matchings[tool_call['tool']]
        arguments = tool_call['arguments']
        print(tool(*arguments))
else:
    print(reply_json)
    tool = tool_matchings[reply_json['tool']]
    arguments = reply_json['arguments']
    print(tool(*arguments)) 

{                                              }
{'put_text_in_file': <function put_text_in_file at 0x7f7dec1296c0>, 'create_directory': <function create_directory at 0x7f7ddcdbbd00>}
{}


KeyError: 'tool'